> **tomato juice dataset**
<br>` 'quality' is the target feature for classification `
<br>` the other features are chemical properties of our product `

**Import the main libraries**

In [ ]:
import numpy as np
import pandas as pd

from time import time

_import the local library_

In [ ]:
# add parent folder path where lib folder is
import sys
if ".." not in sys.path:import sys; sys.path.insert(0, '..') 

In [ ]:
from mylib import show_labels_dist, show_metrics, bias_var_metrics

**Import the Dataset**

In [ ]:
## file path: windows style
df = pd.read_csv('..\\datasets\\tomatjus.csv')

## file path: unix style
#df = pd.read_csv('../datasets/tomatjus.csv')

# shape method gives the dimensions of the dataset
print('Dataset dimensions: {} rows, {} columns'.format(
    df.shape[0], df.shape[1]))

In [ ]:
df.info()

***
**Data Preparation and EDA** (unique to this dataset)
* _Check for missing values_
* _Quick visual check of unique values_
* _Split the classification feature out of the dataset_
* _Check column names of categorical attributes ( for get_dummies() )_
* _Check column names of numeric attributes ( for Scaling )_

**_Let's skip the checking_**

**Classification target feature**
<br>_Make it a multi-class problem, using text labels_

In [ ]:
##  divide into classes by giving a range for quality
##  Make it a multi-class problem: {3,4,5} {6} {7.8}
bins = (2, 5, 6, 8)
group_names = ['Average', 'Premium', 'Special']
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names)

* Split the classification feature out of the dataset 

In [ ]:
## Feature being predicted ("the Right Answer")
labels_col = 'quality'
y = df[labels_col]

## Features used for prediction 
# pandas has a lot of rules about returning a 'view' vs. a copy from slice
# so we force it to create a new dataframe 
X = df.copy()
X.drop(labels_col, axis=1, inplace=True)

***
**<br>Create Test // Train Datasets**
> Split X and y datasets into Train and Test subsets,<br>keeping relative proportions of each class (stratify)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=50, 
                                                    stratify=y)
# train_test_split does random selection, 
#      so we should reset the dataframe indexes
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

**<br>Scaling** comes _after_ test // train split

In [ ]:
numeri = X.select_dtypes(include=['float64','int64']).columns
print(numeri.to_list())

In [ ]:
# scaling the Numeric columns 
# StandardScaler range: -1 to 1, MinMaxScaler range: zero to 1

# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# sklearn docs say 
#   "Don't cheat - fit only on training data, then transform both"
#   fit() expects 2D array: reshape(-1, 1) for single col or (1, -1) single row

for i in numeri:
    arr = np.array(X_train[i])
    scale = MinMaxScaler().fit(arr.reshape(-1, 1))
    X_train[i] = scale.transform(arr.reshape(len(arr),1))

    arr = np.array(X_test[i])
    X_test[i] = scale.transform(arr.reshape(len(arr),1))

**<br>Classifier Selection**

In [ ]:
# prepare list
models = []

##  --  Linear  --  ## 
from sklearn.linear_model import LogisticRegression 
models.append (("LogReg",LogisticRegression())) 
#from sklearn.linear_model import SGDClassifier 
#models.append (("StocGradDes",SGDClassifier())) 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
models.append(("LinearDA", LinearDiscriminantAnalysis())) 
#from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
#models.append(("QuadraticDA", QuadraticDiscriminantAnalysis())) 

##  --  Support Vector  --  ## 
#from sklearn.svm import SVC 
#models.append(("SupportVectorClf", SVC())) 
#from sklearn.svm import LinearSVC 
#models.append(("LinearSVC", LinearSVC())) 
#from sklearn.linear_model import RidgeClassifier
#models.append (("RidgeClf",RidgeClassifier())) 

##  --  Non-linear  --  ## 
#from sklearn.tree import DecisionTreeClassifier 
#models.append (("DecisionTree",DecisionTreeClassifier())) 
#from sklearn.naive_bayes import GaussianNB 
#models.append (("GaussianNB",GaussianNB())) 
#from sklearn.neighbors import KNeighborsClassifier 
#models.append(("K-NNeighbors", KNeighborsClassifier())) 

##  --  Ensemble: bagging  --  ## 
#from sklearn.ensemble import RandomForestClassifier 
#models.append(("RandomForest", RandomForestClassifier())) 
##  --  Ensemble: boosting  --  ## 
#from sklearn.ensemble import AdaBoostClassifier 
#models.append(("AdaBoost", AdaBoostClassifier())) 
#from sklearn.ensemble import GradientBoostingClassifier 
#models.append(("GradientBoost", GradientBoostingClassifier())) 

##  --  NeuralNet (simplest)  --  ## 
#from sklearn.neural_network import MLPClassifier 
#models.append(("MultiLayerPtron", MLPClassifier())) 

print(models)

**<br>Target Label Distributions** (standard block)

In [ ]:
# from our local library
show_labels_dist(X_train,X_test,y_train,y_test)

**<br>Fit and Predict** (standard block)

In [ ]:
# evaluate each model in turn
results = []

print('macro average: unweighted mean per label')
print('weighted average: support-weighted mean per label')
print('MCC: correlation between prediction and ground truth')
print('     (+1 perfect, 0 random prediction, -1 inverse)\n')

for name, clf in models:
    trs = time()
    print('Confusion Matrix:', name)
    
    clf.fit(X_train, y_train)
    ygx = clf.predict(X_test)
    results.append((name, ygx))
    
    tre = time() - trs
    print ("Run Time {} seconds".format(round(tre,2)) + '\n')
    
# Easy way to ensure that the confusion matrix rows and columns
#   are labeled exactly as the classifier has coded the classes
#   [[note the _ at the end of clf.classes_ ]]

    show_metrics(y_test, ygx, clf.classes_)   # from our local library
    print('\nParameters: ', clf.get_params(), '\n\n')

**Bias - Variance Decomposition** (standard block)

In [ ]:
# from our local library
# reduce (cross-validation) folds for faster results
folds = 20
for name, clf in models:
    print('Bias // Variance Decomposition:', name)
    bias_var_metrics(X_train,X_test,y_train,y_test,clf,folds)

***

***
**Statistical Comparison of Models**
<br>The the null hypothesis statement:
>H0: Both models perform equally well on the dataset.
<br>H1: Both models do not have the same performance on the dataset.

Chosen significance threshold is 0.05 (a=0.05) for rejecting the null hypothesis.
***

In [ ]:
# Use the results from the first test run
#     (make sure these match up)

# ('LogReg', LogisticRegression())
model1 = models[0][1]
scores1 = results[0][1]

# ('LinearDA', LinearDiscriminantAnalysis())
model2 = models[1][1]
scores2 = results[1][1]

* Wilcoxon signed-rank test

In [ ]:
from scipy.stats import wilcoxon

# this one requires numeric labels
from sklearn.preprocessing import LabelEncoder
ns1 = LabelEncoder().fit_transform(scores1)
ns2 = LabelEncoder().fit_transform(scores2)

s, pv = wilcoxon(ns1, ns2, zero_method='zsplit')

print('LDA and LR - Wilcoxon Test: p value =', round(pv,3))

if pv <= 0.05:
    print('Since p < 0.05, we can reject the null-hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We may conclude that the difference in performance is significant.')
else:
    print('Since p > 0.05, we cannot reject the null hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We must conclude that the difference in performance is not significant')

* McNemar's test

In [ ]:
from mlxtend.evaluate import mcnemar_table, mcnemar

tb = mcnemar_table(y_target=y_test, 
                   y_model1=scores1, 
                   y_model2=scores2)
chisq, pv = mcnemar(ary=tb, exact=True)

print('LDA and LR - McNemar Test: p value =', round(pv,3), 'and ChiSquare =', chisq)

if pv <= 0.05:
    print('Since p < 0.05, we can reject the null-hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We may conclude that the difference in performance is significant.')
else:
    print('Since p > 0.05, we cannot reject the null hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We must conclude that the difference in performance is not significant')

* 5x2CV paired t-test

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv

# Calculate p-value using previous k-Fold classifier variables
t, pv = paired_ttest_5x2cv(estimator1=model1,
                          estimator2=model2,
                          X=X_train, y=y_train,
                          random_seed=11)

print('LDA and LR - 5x2CV paired t-test: p value =', round(pv,3), 'and t value =', round(t,3))

if pv <= 0.05:
    print('Since p < 0.05, we can reject the null-hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We may conclude that the difference in performance is significant.')
else:
    print('Since p > 0.05, we cannot reject the null hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We must conclude that the difference in performance is not significant')

* 5x2CV combined F test

In [ ]:
from mlxtend.evaluate import combined_ftest_5x2cv

# Calculate p-value using previous k-Fold classifier variables
f, pv = combined_ftest_5x2cv(estimator1=model1,
                            estimator2=model2,
                            X=X_train, y=y_train,
                            random_seed=11)

print('LDA and LR - 5x2CV Combined f Test: p value =', round(pv,3), 'and f value =', round(f,3))

if pv <= 0.05:
    print('Since p < 0.05, we can reject the null-hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We may conclude that the difference in performance is significant.')
else:
    print('Since p > 0.05, we cannot reject the null hypothesis that')
    print('                both models perform equally well on this dataset.')
    print('We must conclude that the difference in performance is not significant')